In [223]:
import pandas as pd
import numpy as np

# Load the train_label
CSV_PATH = './IPEO_Planet_project/train_labels.csv'
df = pd.read_csv(CSV_PATH, sep = ';')
df.fillna(value=0, inplace = True)
df.head()

,image_name,tags,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,train_0,haze,primary,0,0,0,0,0,0,0
1,train_1,agriculture,clear,primary,water,0,0,0,0,0
2,train_2,clear,primary,0,0,0,0,0,0,0
3,train_3,clear,primary,0,0,0,0,0,0,0
4,train_4,agriculture,clear,habitation,primary,road,0,0,0,0


In [224]:
# Create DataFrame containing the labels only
df_drop_train = df.drop('image_name', axis=1)

In [225]:
# Get all the unique labels present in dataset
df_concat = pd.concat([df_drop_train[col] for col in df_drop_train])
label = df_concat.unique()

# Add the label 'image_name'
# Drop the label 0 (corresponding to NaN value in orginial train_label csv)
# Reorder the labels to get athmospheric labels first, ground labels ('agriculture') after
label = label[(label!=0) & (label!='agriculture')]
label = np.insert(label, 0, 'image_name')
label = np.insert(label, 5, 'agriculture')

In [226]:
# Create a Pandas DataFrame with columns corresponding to labels
df_label = pd.DataFrame(columns=label)

In [227]:
# Fill the DataFrame df_label with image_name corresponding to orginial dataframe (df_na)
# Fill with 1 if the label is present in a given row
df_na = pd.read_csv(CSV_PATH, sep = ';')
for index in df_na.index:
    # Add new empty new row in df_label
    df_label = df_label.append(pd.Series(dtype='object'), ignore_index=True)
    
    # Get all the non-nan values of originial df_na
    index_values = df_na.loc[index].dropna().values
    
    # Add the image_name
    index_image_name = index_values[0]
    df_label.image_name[index] = index_image_name
    
    index_labels = index_values[1:]
    for labels in index_labels:
        df_label[labels][index] = 1   

In [228]:
df_label.head()

,image_name,haze,clear,cloudy,partly_cloudy,agriculture,conventional_mine,bare_ground,artisinal_mine,blooming,blow_down,cultivation,habitation,water,primary,road,selective_logging,slash_burn
0,train_0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
1,train_1,NaN,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN
2,train_2,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
3,train_3,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
4,train_4,NaN,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,NaN,NaN


In [240]:
# Fill NaN values with 0
# Set image_name as index
df_label = df_label.fillna(0)
df_label.set_index(df_label.image_name, inplace = True)
df_label.drop(columns='image_name', inplace =True)

In [241]:
# Save DataFrame as csv file
NEW_PATH = './IPEO_Planet_project/new_train_labels.csv'
df_label.to_csv(path_or_buf=NEW_PATH, sep=',')